In [1]:
import osmnx as ox
from shapely.geometry import LineString, Point
from shapely.ops import nearest_points
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.neighbors import NearestNeighbors


In [2]:
from shapely.geometry import GeometryCollection, LineString, MultiLineString
from shapely.ops import linemerge
from shapely.geometry import MultiLineString
from shapely.geometry import Point, Polygon, MultiPolygon

In [3]:
msa = gpd.read_file("/Volumes/Xi/stephan/tl_2019_us_cbsa/tl_2019_us_cbsa.shp")
msa = msa.to_crs(epsg=4326)
mask = msa['NAMELSAD'].str.contains('Micro Area', na=False)
msa = msa[~mask]
msa.loc[msa['NAME'] == 'Louisville/Jefferson County, KY-IN', 'NAME'] = 'Louisville_Jefferson County, KY-IN'
cities=list(msa['NAME'])

In [4]:
cities

['Athens-Clarke County, GA',
 'Atlanta-Sandy Springs-Alpharetta, GA',
 'Atlantic City-Hammonton, NJ',
 'Auburn-Opelika, AL',
 'Augusta-Richmond County, GA-SC',
 'Austin-Round Rock-Georgetown, TX',
 'Bakersfield, CA',
 'Baltimore-Columbia-Towson, MD',
 'Bangor, ME',
 'Barnstable Town, MA',
 'Baton Rouge, LA',
 'Battle Creek, MI',
 'Bay City, MI',
 'Beaumont-Port Arthur, TX',
 'Beckley, WV',
 'Bellingham, WA',
 'Bend, OR',
 'Billings, MT',
 'Binghamton, NY',
 'Birmingham-Hoover, AL',
 'Bismarck, ND',
 'Blacksburg-Christiansburg, VA',
 'Bloomington, IL',
 'Bloomington, IN',
 'Bloomsburg-Berwick, PA',
 'Boise City, ID',
 'Boston-Cambridge-Newton, MA-NH',
 'Boulder, CO',
 'Bowling Green, KY',
 'Bremerton-Silverdale-Port Orchard, WA',
 'Bridgeport-Stamford-Norwalk, CT',
 'Brownsville-Harlingen, TX',
 'Brunswick, GA',
 'Buffalo-Cheektowaga, NY',
 'Cincinnati, OH-KY-IN',
 'Clarksville, TN-KY',
 'Cleveland, TN',
 'Cleveland-Elyria, OH',
 "Coeur d'Alene, ID",
 'College Station-Bryan, TX',
 'Colo

In [4]:
import os

def get_file_names(directory):
    file_names = []
    # 遍历文件夹中的所有文件
    for file_name in os.listdir(directory):
        # 确保是文件，而不是子目录
        if os.path.isfile(os.path.join(directory, file_name)):
            # 获取文件名，不包括扩展名
            name = os.path.splitext(file_name)[0]
            modified_name = name[4:]
            file_names.append(modified_name)
    return file_names

# 替换为你的文件夹路径
directory = '/Volumes/Xi兔/POI+street/TIN_AOI'
file_names = get_file_names(directory)
print(file_names)

['Abilene, TX', 'Akron, OH', 'Albany, GA', 'Albany-Lebanon, OR', 'Albany-Schenectady-Troy, NY', 'Albuquerque, NM', 'Alexandria, LA', 'Allentown-Bethlehem-Easton, PA-NJ', 'Altoona, PA', 'Amarillo, TX', 'Ames, IA', 'Ann Arbor, MI', 'Anniston-Oxford, AL', 'Appleton, WI', 'Asheville, NC', 'Athens-Clarke County, GA', 'Atlanta-Sandy Springs-Alpharetta, GA', 'Atlantic City-Hammonton, NJ', 'Auburn-Opelika, AL', 'Augusta-Richmond County, GA-SC', 'Austin-Round Rock-Georgetown, TX', 'Bakersfield, CA', 'Baltimore-Columbia-Towson, MD', 'Bangor, ME', 'Barnstable Town, MA', 'Baton Rouge, LA', 'Battle Creek, MI', 'Bay City, MI', 'Beaumont-Port Arthur, TX', 'Beckley, WV', 'Bellingham, WA', 'Bend, OR', 'Billings, MT', 'Binghamton, NY', 'Birmingham-Hoover, AL', 'Bismarck, ND', 'Blacksburg-Christiansburg, VA', 'Bloomington, IL', 'Bloomington, IN', 'Bloomsburg-Berwick, PA', 'Boise City, ID', 'Boston-Cambridge-Newton, MA-NH', 'Boulder, CO', 'Bowling Green, KY', 'Bremerton-Silverdale-Port Orchard, WA', 'Brid

In [5]:
len(file_names)

367

In [6]:
def strore_edegs(edges):
    # 创建一个字典来存储每个 (u, v) 对应的顶点
    street_vertices = {}
    # 遍历每个街道段，提取其顶点并按 (u, v) 存储
    for idx, row in edges.iterrows():
       u, v = idx[0], idx[1]  # 获取 u 和 v 作为索引
       if (u, v) not in street_vertices:
        street_vertices[(u, v)] = []
       if row.geometry.geom_type == 'LineString':
        street_vertices[(u, v)].extend(list(row.geometry.coords))
       elif row.geometry.geom_type == 'MultiLineString':
         for subline in row.geometry:
            street_vertices[(u, v)].extend(list(subline.coords))
    # 将所有顶点和对应的 (u, v) 变成两个列表
    all_vertices = []
    vertex_to_uv = []
    for uv, vertices in street_vertices.items():
      all_vertices.extend(vertices)
      vertex_to_uv.extend([uv] * len(vertices))
    # 将顶点列表转换为 numpy 数组用于 KNN
    all_vertices = np.array(all_vertices)
    return street_vertices,all_vertices,vertex_to_uv




In [7]:
def get_road_segment(uv):
    # 过滤匹配的 (u, v) 对
    matching_rows = edges_reset.loc[edges_reset['find_road'] == uv]
    
    # 如果存在匹配的行，返回第一个匹配行的 geometry
    if not matching_rows.empty:
        return matching_rows.iloc[0]['geometry_edge']
    else:
        return None

In [8]:
def get_final_aoi(cluster,edges,edges_reset,vertex_to_uv):
    cluster_aoi_dict={}
    for index,row in cluster.iterrows():
        k = row['cluster_id']
        multipoint = row['geometry']
        points = [Point(point) for point in multipoint.geoms]
        cluster_poi = gpd.GeoDataFrame({'geometry_poi': points})
        cluster_poi = cluster_poi.set_geometry('geometry_poi')
        cluster_poi = cluster_poi.set_crs(edges.crs, allow_override=True)
        cluster_poi_with_edges = cluster_poi.sjoin_nearest(
            edges, how='left', distance_col='distance')
        cluster_poi_with_edges = cluster_poi_with_edges.rename(columns={'index_right0': 'u', 'index_right1': 'v'})
        cluster_poi_with_edges = cluster_poi_with_edges.merge(
           edges_reset[['u', 'v', 'geometry_edge']],
           on=['u', 'v'],
           how='left')
        # 将每个点投影到其最近的边上
        cluster_poi_with_edges['projected_point'] = cluster_poi_with_edges.apply(
        lambda row: row['geometry_edge'].interpolate(
            row['geometry_edge'].project(row['geometry_poi'])
        ),
        axis=1)
        projected_coords = np.array([
        (geom.x, geom.y) for geom in cluster_poi_with_edges['projected_point']])
        # 找到每个投影点最近的顶点
        distances, indices = neighbors.kneighbors(projected_coords)
        nearest_vertex_indices = indices.flatten()
        cluster_poi_with_edges['nearest_vertex_uv'] = [vertex_to_uv[idx] for idx in nearest_vertex_indices]
        cluster_poi_with_edges['road_segment'] = cluster_poi_with_edges['nearest_vertex_uv'].apply(get_road_segment)
        # 收集所有涉及的边缘
        edges_in_cluster = cluster_poi_with_edges['geometry_edge'].unique()
        # 合并这些边为最终的 AOI
        final_aoi = linemerge(MultiLineString(edges_in_cluster))
        #print(final_aoi)
        # 将结果保存到 cluster_aoi_dict 中
        cluster_aoi_dict[k] = final_aoi
    return cluster_aoi_dict
    
    





In [9]:
eval=['Chicago-Naperville-Elgin, IL-IN-WI',
      'Houston-The Woodlands-Sugar Land, TX',
      'Minneapolis-St. Paul-Bloomington, MN-WI',
      'Los Angeles-Long Beach-Anaheim, CA',
      'Miami-Fort Lauderdale-Pompano Beach, FL',
      'New York-Newark-Jersey City, NY-NJ-PA',
      'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
      'Phoenix-Mesa-Chandler, AZ',
       'San Francisco-Oakland-Berkeley, CA',
       'Seattle-Tacoma-Bellevue, WA',
       'Tampa-St. Petersburg-Clearwater, FL',
       'Washington-Arlington-Alexandria, DC-VA-MD-WV'

]

In [12]:
for city in cities[295:]:
   # 读取 graphml 格式的街道数据
   print(city)
   if city in eval:
       continue
   if city in file_names:
       continue
   graph = ox.load_graphml(f'/Volumes/Xi兔/osmnx/msa_graphml/{city}.graphml')
   # 将图形中的街道段（边）转换为 GeoDataFrame
   edges = ox.graph_to_gdfs(graph, nodes=False)
   street_vertices,all_vertices,vertex_to_uv = strore_edegs(edges)
   print('street extract done')
   neighbors = NearestNeighbors(n_neighbors=1)
   neighbors.fit(all_vertices)
   print('pre set KNN')
   edges = edges.rename(columns={'geometry': 'geometry_edge'})
   # 设置主动的 geometry 列
   edges = edges.set_geometry('geometry_edge')
   edges_reset = edges.reset_index()
   edges_reset['find_road'] = list(zip(edges_reset['u'], edges_reset['v'])) 
   cluster=gpd.read_file(f'/Volumes/Xi兔/POI+street/TIN Cluster result/{city}_TIN_cluster.shp')
   print('get cluster')
   cluster_aoi_dict=get_final_aoi(cluster,edges,edges_reset,vertex_to_uv)
   aoi_gdf = gpd.GeoDataFrame(list(cluster_aoi_dict.items()), columns=['cluster', 'geometry'])
   aoi_gdf.to_csv(f'/Volumes/Xi兔/POI+street/TIN_AOI/AOI_{city}.csv')


Roanoke, VA
Rochester, MN
Rochester, NY
Rockford, IL
Rocky Mount, NC
Rome, GA
Sacramento-Roseville-Folsom, CA
Saginaw, MI
St. Cloud, MN
St. George, UT
St. Joseph, MO-KS
St. Louis, MO-IL
Salem, OR
Salinas, CA
Salisbury, MD-DE
Salt Lake City, UT
San Angelo, TX
San Antonio-New Braunfels, TX
San Diego-Chula Vista-Carlsbad, CA
San Francisco-Oakland-Berkeley, CA
San Jose-Sunnyvale-Santa Clara, CA
San Luis Obispo-Paso Robles, CA
Santa Cruz-Watsonville, CA
Santa Fe, NM
Santa Rosa-Petaluma, CA
Savannah, GA
Scranton--Wilkes-Barre, PA
Seattle-Tacoma-Bellevue, WA
Sebastian-Vero Beach, FL
Sebring-Avon Park, FL
Santa Maria-Santa Barbara, CA
Sheboygan, WI
Sherman-Denison, TX
Shreveport-Bossier City, LA
Sierra Vista-Douglas, AZ
Sioux City, IA-NE-SD
Sioux Falls, SD
South Bend-Mishawaka, IN-MI
Spartanburg, SC
Spokane-Spokane Valley, WA
Springfield, IL
Springfield, MA
Springfield, MO
Springfield, OH
State College, PA
Staunton, VA
Stockton, CA
Sumter, SC
Syracuse, NY
Tallahassee, FL
Tampa-St. Petersburg-C

In [ ]:
for city in ['Minneapolis-St. Paul-Bloomington, MN-WI']:
   # 读取 graphml 格式的街道数据
   print(city)
   if city in eval:
       continue
   graph = ox.load_graphml(f'/Volumes/Xi兔/road_graphml/{city}.graphml')
   # 将图形中的街道段（边）转换为 GeoDataFrame
   edges = ox.graph_to_gdfs(graph, nodes=False)
   street_vertices,all_vertices,vertex_to_uv = strore_edegs(edges)
   print('street extract done')
   neighbors = NearestNeighbors(n_neighbors=1)
   neighbors.fit(all_vertices)
   print('pre set KNN')
   edges = edges.rename(columns={'geometry': 'geometry_edge'})
   # 设置主动的 geometry 列
   edges = edges.set_geometry('geometry_edge')
   edges_reset = edges.reset_index()
   edges_reset['find_road'] = list(zip(edges_reset['u'], edges_reset['v'])) 
   cluster=gpd.read_file(f'/Volumes/Xi兔/POI+street/TIN Cluster result/{city}_TIN_cluster.shp')
   print('get cluster')
   cluster_aoi_dict=get_final_aoi(cluster,edges,edges_reset,vertex_to_uv)
   aoi_gdf = gpd.GeoDataFrame(list(cluster_aoi_dict.items()), columns=['cluster', 'geometry'])
   aoi_gdf.to_csv(f'/Volumes/Xi兔/POI+street/TIN_AOI/AOI_{city}.csv')